Precursors!

In [1]:
import os, subprocess

if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)                

In [3]:
if not os.path.isdir('models/heart'):
    os.mkdir('models/heart')
if not os.path.isfile('models/heart/model_best.tf.meta'):
    subprocess.call('curl -o models/heart/model_best.tf.index https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.index', shell=True)
    subprocess.call('curl -o models/heart/model_best.tf.meta https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.meta', shell=True)
    subprocess.call('curl -o models/heart/model_best.tf.data-00000-of-00001 https://storage.googleapis.com/basenji_tutorial_data/model_best.tf.data-00000-of-00001', shell=True)

In [26]:
samples_out = open('data/heart_wigs_index.txt', 'w')
print('0\tCNhs11760\tdata/CNhs11760.bw\taorta', file=samples_out)
print('1\tCNhs12843\tdata/CNhs12843.bw\tartery', file=samples_out)
print('2\tCNhs12856\tdata/CNhs12856.bw\tpulmonic valve', file=samples_out)
samples_out.close()

Analyzing noncoding variation associated with disease is a major application of Basenji. I now offer several tools to enable that analysis. If you have a small set of variants and know what datasets are most relevant, [basenji_sat_vcf.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_vcf.py) lets you perform a saturation mutagenesis of the variant and surrounding region to see the relevant nearby motifs.

If you want scores measuring the influence of those variants on all datasets,
 * [basenji_sad.py](https://github.com/calico/basenji/blob/master/bin/basenji_sad.py) computes my SNP activity difference (SAD) score--the predicted change in aligned fragments to the region.
 * [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py) computes my SNP expression difference (SED) score--the predicted change in aligned fragments to gene TSS's.

Here, I'll demonstrate those two programs. You'll need
 * Trained model
 * Input file (FASTA or HDF5 with test_in/test_out)

First, you can either train your own model in the [Train/test tutorial](https://github.com/calico/basenji/blob/master/tutorials/train_test.ipynb) or use one that I pre-trained from the models subdirectory.

As an example, we'll study a prostate cancer susceptibility allele of rs339331 that increases RFX6 expression by modulating HOXB13 chromatin binding (http://www.nature.com/ng/journal/v46/n2/full/ng.2862.html).

First, we'll use [basenji_sad.py](https://github.com/calico/basenji/blob/master/bin/basenji_sad.py) to predict across the region for each allele and compute stats about the mean and max differences.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -f | data/hg19.ml.fa | Genome fasta. |
| -g | data/human.hg19.genome | Genome assembly chromosome length to bound gene sequences. |
| -l | 131072 | Saturation mutagenesis region in the center of the given sequence(s) |
| -o | rfx6_sad | Outplot plot directory. |
| --rc | | Predict forward and reverse complement versions and average the results. |
| -t | data/heart_wigs.txt | Target labels. |
| params_file | models/params_small.txt | Table of parameters to setup the model architecture and optimization parameters. |
| model_file | models/heart/model_best.tf | Trained saved model prefix. |
| vcf_file | data/rs339331.vcf | VCF file specifying variants to score. |

In [6]:
! basenji_sad.py -f data/hg19.ml.fa -g data/human.hg19.genome -l 131072 -o output/rfx6_sad --rc -t data/heart_wigs_index.txt models/params_small.txt models/heart/model_best.tf data/rs339331.vcf

/Users/davidkelley/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
{'optimizer': 'adam', 'cnn_filters': [128, 128, 192, 256, 256, 32, 32, 32, 32, 32, 32, 384], 'target_pool': 128, 'learning_rate': 0.002, 'link': 'softplus', 'batch_size': 4, 'cnn_filter_sizes': [20, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 1], 'cnn_dilation': [1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'cnn_pool': [2, 4, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0], 'num_targets': 3, 'batch_buffer': 4096, 'adam_beta2': 0.98, 'loss': 'poisson', 'cnn_dense': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0], 'cnn_dropout': 0.1, 'adam_beta1': 0.97}
Targets pooled by 128 to length 1024
Convolution w/ 3 384x1 filters to final targets
Model building time 31.992178
2018-05-16 14:58:26.591667: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was n

rfx6_sad/sad_table.txt now contains a table describing the results.

The *u* in *upred* and *usad* refers to taking the mean across the sequence, whereas *x* in *xpred* and *xsad* refers to the maximum position. 
Then *sad* refers to subtracting the alt allele prediction from the ref allele, and *sar* refers to adding a pseudocount 1 and taking log2 of their ratio.

In [9]:
! cat output/rfx6_sad/sad_table.txt

rsid ref alt ref_pred alt_pred sad sar geo_sad ref_lpred alt_lpred lsad lsar ref_xpred alt_xpred xsad xsar target_index target_id target_label
rs339331           T      C |   258.35   258.22   -0.128 -0.0007  -0.147 |   1.461   1.426  -0.035 -0.0208 |   0.241   0.230  -0.011 -0.0129 |    0    CNhs11760 aorta
rs339331           T      C |   103.91   103.88   -0.034 -0.0005  -0.042 |   0.368   0.362  -0.006 -0.0062 |   0.067   0.061  -0.005 -0.0069 |    1    CNhs12843 
rs339331           T      C |   465.86   465.76   -0.095 -0.0003  -0.093 |   2.001   1.986  -0.016 -0.0075 |   0.334   0.319  -0.015 -0.0160 |    2    CNhs12856 pulmonic valv


We can sort by *xsar* to get an idea of the datasets where Basenji sees the largest difference between the two alleles.

In [10]:
! sort -k13 -g output/rfx6_sad/sad_table.txt

rsid ref alt ref_pred alt_pred sad sar geo_sad ref_lpred alt_lpred lsad lsar ref_xpred alt_xpred xsad xsar target_index target_id target_label
rs339331           T      C |   258.35   258.22   -0.128 -0.0007  -0.147 |   1.461   1.426  -0.035 -0.0208 |   0.241   0.230  -0.011 -0.0129 |    0    CNhs11760 aorta
rs339331           T      C |   465.86   465.76   -0.095 -0.0003  -0.093 |   2.001   1.986  -0.016 -0.0075 |   0.334   0.319  -0.015 -0.0160 |    2    CNhs12856 pulmonic valv
rs339331           T      C |   103.91   103.88   -0.034 -0.0005  -0.042 |   0.368   0.362  -0.006 -0.0062 |   0.067   0.061  -0.005 -0.0069 |    1    CNhs12843 


In [11]:
! sort -k13 -gr output/rfx6_sad/sad_table.txt

rs339331           T      C |   103.91   103.88   -0.034 -0.0005  -0.042 |   0.368   0.362  -0.006 -0.0062 |   0.067   0.061  -0.005 -0.0069 |    1    CNhs12843 
rs339331           T      C |   465.86   465.76   -0.095 -0.0003  -0.093 |   2.001   1.986  -0.016 -0.0075 |   0.334   0.319  -0.015 -0.0160 |    2    CNhs12856 pulmonic valv
rs339331           T      C |   258.35   258.22   -0.128 -0.0007  -0.147 |   1.461   1.426  -0.035 -0.0208 |   0.241   0.230  -0.011 -0.0129 |    0    CNhs11760 aorta
rsid ref alt ref_pred alt_pred sad sar geo_sad ref_lpred alt_lpred lsad lsar ref_xpred alt_xpred xsad xsar target_index target_id target_label


These are inconclusive small effect sizes, not surprising given that we're only studying heart CAGE. The proper cell types and experiments would shed more light.

Alternatively, we can directly query the predictions at gene TSS's using [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py).

[basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py) takes as input the gene sequence HDF5 format described in [genes.ipynb](https://github.com/calico/basenji/blob/master/tutorials/genes.ipynb). There's no harm to providing an HDF5 that describes all genes, but it's too big to easily move around so I constructed one that focuses on RFX6.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -g | data/human.hg19.genome | Genome assembly chromosome length to bound gene sequences. |
| -o | rfx6_sed | Outplot plot directory. |
| --rc | | Predict forward and reverse complement versions and average the results. |
| -w | 128 | Sequence bin width at which predictions are made. |
| params_file | models/params_med.txt | Table of parameters to setup the model architecture and optimization parameters. |
| model_file | models/gm12878.tf | Trained saved model prefix. |
| genes_hdf5_file | data/rfx6.h5 | HDF5 file specifying gene sequences to query. |
| vcf_file | data/rs339331.vcf | VCF file specifying variants to score. |

Before running [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py), we need to generate an input data file for RFX6. Using an included GTF file that contains only RFX6, one can use [basenji_hdf5_genes.py]((https://github.com/calico/basenji/blob/master/bin/basenji_hdf5_genes.py) to create the required format.

In [12]:
! basenji_hdf5_genes.py -g data/human.hg19.genome -l 131072 -c 0.333 -w 128 data/hg19.ml.fa data/rfx6.gtf data/rfx6.h5

In [23]:
! basenji_sed.py -a -g data/human.hg19.genome -o output/rfx6_sed --rc models/params_small.txt models/heart/model_best.tf data/rfx6.h5 data/rs339331.vcf

/Users/davidkelley/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Intersecting gene sequences with SNPs...1 sequences w/ SNPs
{'target_pool': 128, 'loss': 'poisson', 'num_targets': 3, 'cnn_dropout': 0.1, 'cnn_filters': [128, 128, 192, 256, 256, 32, 32, 32, 32, 32, 32, 384], 'cnn_pool': [2, 4, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0], 'optimizer': 'adam', 'adam_beta2': 0.98, 'cnn_filter_sizes': [20, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 1], 'batch_buffer': 4096, 'learning_rate': 0.002, 'batch_size': 4, 'link': 'softplus', 'cnn_dilation': [1, 1, 1, 1, 1, 2, 4, 8, 16, 32, 64, 1], 'adam_beta1': 0.97, 'cnn_dense': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]}
Targets pooled by 128 to length 1024
Convolution w/ 3 384x1 filters to final targets
2018-05-16 16:58:00.804860: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions tha

In [24]:
! sort -k9 -g output/rfx6_sed/sed_gene.txt

rsid ref alt gene tss_dist ref_pred alt_pred sed ser target_index target_id target_label
rs339331      T     C ENSG00000185002.9_1  4100  4.6992  4.6992  0.0000  0.0000    1           t1 
rs339331      T     C ENSG00000185002.9_1  4100 11.7578 11.7578  0.0000  0.0000    2           t2 
rs339331      T     C ENSG00000185002.9_1  4100 12.0859 12.0859  0.0000  0.0000    0           t0 


In [25]:
! sort -k9 -gr output/rfx6_sed/sed_gene.txt

rs339331      T     C ENSG00000185002.9_1  4100 12.0859 12.0859  0.0000  0.0000    0           t0 
rs339331      T     C ENSG00000185002.9_1  4100 11.7578 11.7578  0.0000  0.0000    2           t2 
rs339331      T     C ENSG00000185002.9_1  4100  4.6992  4.6992  0.0000  0.0000    1           t1 
rsid ref alt gene tss_dist ref_pred alt_pred sed ser target_index target_id target_label
